## Goal of Analysis:

### 1. answer the specific questions from managers

### 2. build a song recommendation model

### Data Preprocessing

In [1]:
import numpy as np
import pandas as pd

In [2]:
# convert_dates is similar to parse_date in pd.read_csv

data = pd.read_json('song.json', convert_dates=['user_sign_up_date', 'time_played'])
data

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00
...,...,...,...,...,...,...
3995,MKVEXMRJGF,122,Louisiana,2015-05-16,Paperback Writer,2015-06-03 08:26:26
3996,HMBEUWPBQI,76,New York,2015-05-08,Reprise / Day in the Life,2015-06-14 15:46:04
3997,KIYDZUFUJN,68,Florida,2015-05-08,Get Back,2015-06-13 13:13:11
3998,BZNJXQSTTL,138,Alabama,2015-05-17,Help!,2015-06-01 17:01:40


In [3]:
data.dtypes

id                           object
user_id                       int64
user_state                   object
user_sign_up_date    datetime64[ns]
song_played                  object
time_played          datetime64[ns]
dtype: object

In [4]:
data.isnull().sum()

id                   0
user_id              0
user_state           0
user_sign_up_date    0
song_played          0
time_played          0
dtype: int64

### Question 1： What are the top 3 and the bottom 3 states in terms of number of users

In [5]:
# We will need to find the unique ID instead of just summing up the IDs

user_group = data.groupby('user_state')['user_id'].apply(lambda x: len(np.unique(x))).sort_values(ascending=False)

In [6]:
# Top 3:

user_group.head(3)

user_state
New York      23
California    21
Texas         15
Name: user_id, dtype: int64

In [7]:
# Top 3:

user_group.tail(3)

user_state
Nebraska    1
Arizona     1
Kansas      1
Name: user_id, dtype: int64

The top three states are New York, California, and Texas. The last three states are Nebraska, Arizona, and Kansas.

### Question 2: What are the top 3 and the bottom 3 states in terms of user engagement? (which states users are using the product a lot/very little)

#### Here, I define the user engagement as the average time a user play the song in each state

In [8]:
# first define the user_group as a dataframe and rename the column

user_group = user_group.reset_index().rename(columns={'user_id': 'user_count'})
user_group.head()

,user_state,user_count
0,New York,23
1,California,21
2,Texas,15
3,Pennsylvania,9
4,Ohio,9


In [9]:
# create a dataframe for play count per state

state_id_group = data.groupby('user_state')['id'].count().reset_index().rename(columns={'id': 'play_count'})
state_id_group.head()

,user_state,play_count
0,Alabama,104
1,Alaska,58
2,Arizona,22
3,Arkansas,34
4,California,425


In [10]:
# merge user count and play count

state_user_play = pd.merge(left=user_group, right=state_id_group, on='user_state')

In [11]:
# calculate average play
state_user_play['average_play'] = state_user_play['play_count'] / state_user_play['user_count']

In [12]:
# sort according to average_play
state_user_play = state_user_play.sort_values(by='average_play', ascending=False)
state_user_play.head()

,user_state,user_count,play_count,average_play
38,Nebraska,1,36,36.000000
27,Alaska,2,58,29.000000
22,South Carolina,3,85,28.333333
25,Mississippi,3,85,28.333333
37,Rhode Island,1,27,27.000000


In [13]:
# Top 3

state_user_play.head(3)

,user_state,user_count,play_count,average_play
38,Nebraska,1,36,36.000000
27,Alaska,2,58,29.000000
22,South Carolina,3,85,28.333333


In [14]:
# Last 3

state_user_play.tail(3)

,user_state,user_count,play_count,average_play
17,Minnesota,4,42,10.5
30,Virginia,2,17,8.5
40,Kansas,1,8,8.0


So the top three states are Nebraska, Alaska, and South Carolina. The last three are Minnesota, Virginia, and Kansad.

### Question 3: list the first user who signed-up from each state

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [59]:
# filter out the first signup date for easy manipulation later on

first_signup = data[data.groupby('user_state')['user_sign_up_date'].transform('min').eq(data['user_sign_up_date'])]

first_signup
# first_signup.loc[:,['user_id', 'user_state', 'user_sign_up_date']]

,id,user_id,user_state,user_sign_up_date,song_played,time_played
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00
8,HZICITYVPP,174,Rhode Island,2015-05-19,Revolution,2015-06-14 04:55:17
18,FDYRTNRNLB,11,Pennsylvania,2015-05-02,She Loves You,2015-06-05 18:46:03
24,RJAXELQHQP,85,Missouri,2015-05-09,Come Together,2015-06-25 12:23:05
...,...,...,...,...,...,...
3981,GZWDHMNTML,105,Arizona,2015-05-12,Birthday,2015-06-01 13:05:17
3988,QHLNQQZUVK,165,Idaho,2015-05-19,Revolution,2015-06-04 22:59:02
3989,DQXBQLVUSJ,44,California,2015-05-04,Yellow Submarine,2015-06-26 10:36:35
3990,WJPGBHEPUX,178,Iowa,2015-05-19,Yellow Submarine,2015-06-05 15:02:01


In [60]:
# first find out the first signup id for each state

first_signup_id = first_signup.groupby('user_state')['user_id'].apply(lambda x: np.unique(x))

In [61]:
# then find out the first signup time in each state

first_signup_time = first_signup.groupby('user_state')['user_sign_up_date'].apply(lambda x: np.unique(x))

In [62]:
first_signup_id.shape, first_signup_time.shape

((41,), (41,))

In [63]:
# combine the two lists to get our user id and user signup date list

first_signup_list = pd.merge(left = first_signup_id, right = first_signup_time, on='user_state', how='left')
first_signup_list

,user_id,user_sign_up_date
user_state,,
Alabama,[5],[2015-05-01T00:00:00.000000000]
Alaska,[106],[2015-05-12T00:00:00.000000000]
Arizona,[105],[2015-05-12T00:00:00.000000000]
Arkansas,[78],[2015-05-08T00:00:00.000000000]
California,"[39, 44]",[2015-05-04T00:00:00.000000000]
Colorado,"[166, 173]",[2015-05-19T00:00:00.000000000]
Connecticut,[127],[2015-05-16T00:00:00.000000000]
Florida,"[41, 43]",[2015-05-04T00:00:00.000000000]
Georgia,"[16, 20]",[2015-05-02T00:00:00.000000000]


The manager can use this list to give the rewards for people in each state

### Question 4: build a song recommendation model based on the previous songs the user listened

#### There are two common methods for this recommendation problem, one is collaborative recommendation, the other is KNN model, 


#### The idea behind collaborative recommendation is to recommend songs that are similar to the user's past playlist


#### The idea of KNN is to find out the most popular songs among all the users and recommend top 10 popular songs.

#### I will use KNN model in this question.

reference: https://github.com/SwathyMM/Top-10-song-recommendation-using-collaborative-filtering-and-KNN/blob/master/Song%20recommender.ipynb

In [64]:
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [107]:
data.nunique()

id                   4000
user_id               196
user_state             41
user_sign_up_date      20
song_played           100
time_played          3997
dtype: int64

There are 100 unique songs, and 196 unique users.

In [97]:
# first calculate how many times each song played by each user

time_played = data.groupby(['song_played', 'user_id']).count().reset_index().loc[:,('song_played', 'user_id', 'id')]
time_played.rename(columns = {'id':'play_count'}, inplace = True)
time_played

,song_played,user_id,play_count
0,A Day In The Life,3,1
1,A Day In The Life,4,3
2,A Day In The Life,6,2
3,A Day In The Life,12,2
4,A Day In The Life,13,3
...,...,...,...
2767,You Never Give Me Your Money,162,1
2768,You Never Give Me Your Money,179,1
2769,You Never Give Me Your Money,184,1
2770,You're Going To Lose That Girl,69,1


In [106]:
# let me put the songs and user id into a pivot table for easy manipulation

song_pivot=time_played.pivot(index='user_id',columns='song_played',values='play_count').fillna(0)
song_pivot

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,BALLAD OF JOHN AND YOKO,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We put 196 unique users and 100 unique songs into a matrix, the values are the times each user play each songs

#### fit the KNN model

In [145]:
from sklearn.neighbors import NearestNeighbors

knn=NearestNeighbors(n_neighbors=10,metric='cosine')
Model=knn.fit(song_pivot)

In [146]:
user = song_pivot.iloc[1,]

In [147]:
distances, indices = Model.kneighbors([user])
distances, indices

(array([[1.11022302e-16, 2.40565667e-01, 2.81250000e-01, 2.88487526e-01,
         3.20975287e-01, 3.31650408e-01, 3.37410844e-01, 3.62112046e-01,
         3.65208281e-01, 3.75000000e-01]]),
 array([[  1,  93, 187,  79,  73,  42, 175, 167,  19, 138]], dtype=int64))

In [148]:
neighbors=[]
for item in indices[0][1:]:
    neighbors.append(time_played[time_played.index == item].user_id.values[0])
neighbors

[181, 32, 150, 136, 76, 18, 106, 38, 14]

In [149]:
neighbor_songs=pd.DataFrame(columns=['user_id','play_count','song_played'])

for item in neighbors:
    neighbor_songs=neighbor_songs.append(time_played[time_played.user_id == item],ignore_index = True)
neighbor_songs

,user_id,play_count,song_played
0,181,2,A Day In The Life
1,181,1,Back In the U.S.S.R.
2,181,1,Can't Buy Me Love
3,181,2,Come Together
4,181,2,Get Back
...,...,...,...
156,14,1,Sgt. Pepper/with A Little Help From My Friends
157,14,1,The Fool On The Hill
158,14,1,The Long And Winding Road
159,14,1,Twist and Shout


In [150]:
neighbor_songs=pd.DataFrame({'Count':neighbor_songs['play_count'],'song_played':neighbor_songs['song_played'].tolist()})
neighbor_songs

,Count,song_played
0,2,A Day In The Life
1,1,Back In the U.S.S.R.
2,1,Can't Buy Me Love
3,2,Come Together
4,2,Get Back
...,...,...
156,1,Sgt. Pepper/with A Little Help From My Friends
157,1,The Fool On The Hill
158,1,The Long And Winding Road
159,1,Twist and Shout


In [151]:
neighbor_songs=neighbor_songs.sort_values('Count',ascending=False).reset_index(drop=True)
neighbor_songs

,Count,song_played
0,6,Come Together
1,5,Hello Goodbye
2,5,Come Together
3,4,Come Together
4,3,Come Together
...,...,...
156,1,Sgt. Pepper/with A Little Help From My Friends
157,1,Can't Buy Me Love
158,1,Birthday
159,1,She Loves You


In [159]:
Recommended_Song=neighbor_songs['song_played'][0:20]
print(pd.DataFrame(np.unique(Recommended_Song)))

                                                0
0                               A Day In The Life
1                          And Your Bird Can Sing
2                                   Come Together
3                                        Get Back
4                                   Hello Goodbye
5                              Here Comes The Sun
6                                       Let It Be
7                                      Revolution
8  Sgt. Pepper/with A Little Help From My Friends
9                    While My Guitar Gently Weeps


We can recommend those top 10 most popular songs to the users